In [1]:
import os
import sys

import numpy as np
import plotly.graph_objects as go
from plotly.offline.offline import iplot

In [2]:
os.chdir(os.pardir)

In [3]:
from src.performance import get_portfolio_performance
from src.capm import min_var_points

In [4]:
market = "ie00bym11h29"

In [5]:
performance, covs, weights = get_portfolio_performance()

In [6]:
performance

,return,sd,beta
ie00bqn1k901,0.095574,0.086872,1.226779
us0846707026,0.118635,0.056963,0.888975
ie00bym11h29,0.139594,0.042649,1.000000
ie00b5bmr087,0.163259,0.050034,1.036367
ch0131872431,0.116387,0.030923,0.633595
ch0106027193,0.114064,0.015164,0.131576
eur,-0.003096,0.001257,0.053005
usd,-0.024936,0.005958,-0.020396
chf,0.000000,0.000000,0.000000
portfolio,0.032772,0.001098,0.115776


In [7]:
riskfree = 0

In [10]:
to_ignore = ["chf", "eur", "usd", "portfolio"]
min_sigmas, returns, opt_weigths, sig_m, mu_m = min_var_points(
    mus=performance["return"].drop(to_ignore).values, 
    sigma=covs.drop(to_ignore, axis=0).drop(to_ignore, axis=1).values, 
    riskfree=riskfree,
    returns=np.linspace(0.02, 0.22, 100))

In [13]:
fig = go.Figure(layout_title="Capital Market Line",
                layout_xaxis_title="Annualised std deviation [%]",
                layout_yaxis_title="Expected return [%]"
               )
for i in performance.index:
    fig.add_trace(go.Scatter(
        x=[performance["sd"].loc[i] * 100], 
        y=[performance["return"].loc[i] * 100],
        name=i,
        mode="markers"
    ))
fig.add_trace(go.Scatter(
    x=[0, sig_m * 2 * 100],
    y=[riskfree, mu_m * 2 * 100],
    mode="lines",
    name="CML",
))
fig.add_trace(go.Scatter(
    x=min_sigmas * 100,
    y=returns * 100,
    mode="lines",
    name="Minimal Variance Portfolios",
))
iplot(fig)

In [12]:
fig = go.Figure(layout_title="Securities market line",
                layout_xaxis_title="Beta",
                layout_yaxis_title="Expected return"
               )
for i in performance.index:
    fig.add_trace(go.Scatter(
        x=[performance["beta"].loc[i]], 
        y=[performance["return"].loc[i] * 100],
        name=i,
        mode="markers"
    ))
fig.add_trace(go.Scatter(
    x=[0, performance["beta"].loc[market] * 2],
    y=[0, performance["return"].loc[market] * 2 * 100],
    mode="lines",
    name="SML",
))
iplot(fig)